## Import Libraries

In [45]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict, Counter
import nltk
nltk.download('stopwords')
nltk.download('names')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\ishac\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ishac\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Load Dataset

In [20]:
ds = pd.read_csv("C:/Users/ishac/Downloads/Evaluation-dataset.csv",  quoting=0, header=None)
ds

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Tires where delivered to the garage of my choi...,garage service positive,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Easy Tyre Selection Process, Competitive Prici...",garage service positive,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Very easy to use and good value for money.,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Really easy and convenient to arrange,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,It was so easy to select tyre sizes and arrang...,location positive,value for money positive,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10127,"I ordered the wrong tyres, however [REDACTED] ...",refund positive,delivery punctuality positive,refund timescale positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10128,"Good experience, first time I have used [REDAC...",length of fitting positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10129,"I ordered the tyre I needed on line, booked a ...",location positive,delivery punctuality positive,length of fitting positive,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10130,Excellent service from point of order to fitti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## User Input

In [47]:
ab = ds.iloc[0:,0:1]
ab = np.array(ab)
size = ds.shape
size = size[0]
row = int(input(f"enter no of row rangeing from 1 - {size}(: "))
row = ab[row-1]
print(row)
review=str(row)

enter no of row rangeing from 1 - 10132(: 6
["service was excellent. Only slight downside was not knowing exact time at garage although the garage were so quick so I wasn't delayed"]


## Aspect & Problem

In [48]:
aspects = ['garage service', 'wait time', 'booking', 'value', 'money']
problems = ['incorrect', 'missing', 'delay', 'changed']

## Data Preprocessing

In [49]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text

## Tokenization

In [50]:
def tokenize_text(text):
    return text.split()


## Aspect Extraction & Filtering Unwanted Words

In [51]:
def extract_aspects_sentiments(review):
    processed_text = preprocess_text(review)
    tokens = tokenize_text(processed_text)
    aspects_sentiments = defaultdict(list)
   
    unwanted = nltk.corpus.stopwords.words("english")
    unwanted.extend([w.lower() for w in nltk.corpus.names.words()])
  
    def skip_unwanted(pos_tuple):
        word, tag = pos_tuple
        if not word.isalpha() or word in unwanted:
            return False
        if tag.startswith("NN"):
            return False
        return True
    
    positive_words = [word for word, tag in filter(
        skip_unwanted,
        nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["pos"]))
    )]
    negative_words = [word for word, tag in filter(
        skip_unwanted,
        nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["neg"]))
    )]
    
    # Lexicon-based approach for sentiment analysis
    positive_words = nltk.FreqDist(positive_words)
    negative_words = nltk.FreqDist(negative_words)
   
    for aspect in aspects:
        
        if any(word in processed_text for word in aspect.split()):
            
            sentiment = "neutral"
       
            for token in tokens:
                if token in positive_words:
                    sentiment = "positive"
                elif token in negative_words:
                    sentiment = "negative"
            aspects_sentiments[aspect].append(sentiment)
    
    return aspects_sentiments

## Extracting Aspects and Sentiments from a Review

In [52]:
aspects_sentiments = extract_aspects_sentiments(review)

## Summarizing Aspect Sentiments from Reviews

In [53]:
results = []
for aspect, sentiments in aspects_sentiments.items():
    sentiment_count = Counter(sentiments)
    final_sentiment = "negative" if aspect in problems else "positive" if sentiment_count['positive'] > sentiment_count['negative'] else "negative"
    results.append(f"{aspect} {final_sentiment}")


## Result

In [54]:
print("Subtheme Sentiments:")
print("\n".join(results))

Subtheme Sentiments:
garage service positive
wait time positive
